In [8]:
import pickle
import numpy as np
import os
import platform
import numpy as np
import matplotlib.pyplot as plt
import download
import data_utils
%matplotlib inline

In [9]:
class KNN(object):
  def _init_(self, k=1):
   pass
  
  def training(self, X, y):
    self.X_train = X
    self.y_train = y
  
  def predict(self, X, k=1, num_loops=0):
    if num_loops==0:
      distances = self.compute_distances(X)
    else:
        raise ValueError('Invalid value %d for num_loops' % num_loops)

    return self.predict_labels(distances, k=k)

  def compute_distances(self, X):
    num_train = self.X_train.shape[0]
    num_test = X.shape[0]
    distances = np.zeros((num_test, num_train))

    #dists = np.sqrt(np.sum(np.square(self.X_train), axis=1) + np.sum(np.square(X), axis=1)[:, np.newaxis] - 2 * np.dot(X, self.X_train.T))
    # (x-y)^2 = x^2 - 2xy + y^2

    x2 = np.sum(np.square(self.X_train), axis=1)
    y2 = np.sum(np.square(X), axis=1)[:, np.newaxis]
    xy =  np.dot(X, self.X_train.T)
    distances = np.sqrt(x2 - 2*xy + y2)
    pass
    return distances

  def predict_labels(self, dists, k=1):
      num_test = dists.shape[0]
      y_pred = np.zeros(num_test)
      for i in range(num_test):
          closest_y = []
          sorted_dist = np.argsort(dists[i])
          closest_y = list(self.y_train[sorted_dist[0:k]])
          pass
          y_pred[i]= (np.argmax(np.bincount(closest_y)))
          pass
      return y_pred

In [10]:
url = "https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz"
download_dir = "./CIFARdata"
download.maybe_download_and_extract(url,download_dir)

- Download progress: 100.0%
Download finished. Extracting files.
Done.


In [11]:
cifar10_dir = './CIFARdata/cifar-10-batches-py'
X_train, y_train, X_test, y_test = data_utils.load_CIFAR10(cifar10_dir)

# Checking the size of the training and testing data
print('Training data shape: ', X_train.shape)
print('Training labels shape: ', y_train.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

Training data shape:  (50000, 32, 32, 3)
Training labels shape:  (50000,)
Test data shape:  (10000, 32, 32, 3)
Test labels shape:  (10000,)


In [ ]:
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
num_classes = len(classes)
samples_per_class = 7
plt.figure(figsize = (15, 15))
for y, cls in enumerate(classes):
    idxs = np.flatnonzero(y_train == y)
    idxs = np.random.choice(idxs, samples_per_class, replace=False)
    for i, idx in enumerate(idxs):
        plt_idx = i * num_classes + y + 1
        plt.subplot(samples_per_class, num_classes, plt_idx)
        plt.imshow(X_train[idx].astype('uint8'))
        plt.axis('off')
        if i == 0:
            plt.title(cls)
plt.show()

In [ ]:
# Memory error prevention by subsampling data

num_training = 10000
mask = list(range(num_training))
X_train = X_train[mask]
y_train = y_train[mask]

num_test = 1000
mask = list(range(num_test))
X_test = X_test[mask]
y_test = y_test[mask]

In [ ]:
# reshaping data and placing into rows
X_train = np.reshape(X_train, (X_train.shape[0], -1))
X_test = np.reshape(X_test, (X_test.shape[0], -1))
print(X_train.shape, X_test.shape)

In [ ]:
classifier = KNN()
classifier.training(X_train, y_train)
distances= classifier.compute_distances(X_test)
y_test_pred = classifier.predict_labels(distances, k=5)
num_correct = np.sum(y_test_pred == y_test)
accuracy = float(num_correct) / num_test
print('Got %d / %d correct => accuracy: %f' % (num_correct, num_test, accuracy))

In [ ]:
num_folds = 5
k_values = [1, 5, 8, 10, 12, 20, 50, 65, 80, 100]

X_train_folds = []
y_train_folds = []

X_train_folds = np.array_split(X_train,num_folds)
y_train_folds = np.array_split(y_train,num_folds)
k_to_accuracies = {}

for k in k_values:
    k_to_accuracies[k] = []
    for num_knn in range(0,num_folds):
        X_test = X_train_folds[num_knn]
        y_test = y_train_folds[num_knn]
        X_train = X_train_folds
        y_train = y_train_folds
        
        temp = np.delete(X_train,num_knn,0)
        X_train = np.concatenate((temp),axis = 0)
        y_train = np.delete(y_train,num_knn,0)
        y_train = np.concatenate((y_train),axis = 0)
        
        classifier = KNN()
        classifier.training(X_train, y_train)
        dists = classifier.compute_distances(X_test)
        y_test_pred = classifier.predict_labels(dists, k)

        num_correct = np.sum(y_test_pred == y_test)
        accuracy = float(num_correct) / num_test
        k_to_accuracies[k].append(accuracy)


print("Printing 5-fold accuracies for varying values of k:")
print()
for k in sorted(k_to_accuracies):
    for accuracy in k_to_accuracies[k]:
        print('k = %d, accuracy = %f' % (k, accuracy))

In [ ]:
for k in k_values:
    accuracies = k_to_accuracies[k]
    print('k = %d, average accuracy = %f' % (k, np.average(accuracies)))
    plt.scatter([k] * len(accuracies), accuracies)

# plot the trend line with error bars that correspond to standard deviation
accuracies_mean = np.array([np.mean(v) for k,v in sorted(k_to_accuracies.items())])
accuracies_std = np.array([np.std(v) for k,v in sorted(k_to_accuracies.items())])
plt.rcParams['figure.figsize'] = [10,12]
plt.errorbar(k_values, accuracies_mean, yerr=accuracies_std)
plt.title('Cross-validation on k')
plt.xlabel('k')
plt.ylabel('Cross-validation accuracy')
plt.show()